# Evolutionary Computation (CS5048)

Edgar Covantes Osuna, PhD

Email: edgar.covantes@tec.mx

Student name: **Miguel Lara **

ID: 

$
\def\order{{\rm O{\Tiny RDER}}}
\def\mayority{{\rm M{\Tiny AYORITY}}}
\def\sorting{{\rm S{\Tiny ORTING}}}
\def\inv{{\rm INV}}
\def\ham{{\rm HAM}}
\def\run{{\rm RUN}}
\def\las{{\rm LAS}}
\def\exc{{\rm EXC}}
$

## Genetic Programming
From the Lecture "Genetic Programming" you learned the theoretical fundations of genetic programming as a small set of optimisation and black box optimisation problems whose main goal is to evolve a population of computer
programs.

You know are familiar with the terms genetic programming, tree-based and linear genetic programming, including their representation, initialisation and operators such as mutation and crossover. You are also familiar with Automatically Defined Functions.

## Instructions

In this activity you will need to make a literature review of the well-known GPs approaches to tackle 3 fitness functions (all of them are defined as maximisation problems). 

You can use any GP representation you want, and you can use any framework available on the internet. This is, you are free to use any tool available on internet so you do not program your solution from scratch.

A good place to start is to check [DEAP](https://github.com/deap/deap), this is an evolutionary computation framework with GP capabilities.

It is expected from you to read the literature so you can solve these problems. By knowing the problem it will be easier for you to make decisions about parameter settings, mechanisms to apply, the search space to use and the structure of the individuals to use.

**The goal**:

At the end of the execution of your evolutionary strategy you should present at least one individual representing the global optimum and its corresponding fitness value.

**Marking scheme**:

- $\order$ (33.5 points). 
- $\mayority$ (33.5 points).
- $\sorting$
  - $\inv$ (6.6 points).
  - $\ham$ (6.6 points).
  - $\run$ (6.6 points).
  - $\las$ (6.6 points).
  - $\exc$ (6.6 points).

**Suggestions**
- Read carefully each section and spend time understanding each function. Each function may require specific representations or operator. I highly suggest reading the slides and the papers referenced there before solving the functions.

## Imports

You can add all the necessary packages in the following cell.

In [272]:
import numpy as np
import string
import random

!pip install deap

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

!sudo apt-get install graphviz libgraphviz-dev pkg-config
!sudo apt-get install python-pip python-virtualenv
!pip install pygraphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-idna python-ipaddress python-keyring
  python-keyrings.alt python-pip-whl python-pkg-resources python-secretstorage
  python-setuptools python-six python-wheel python-xdg python3-pkg-resources
  python3-virtualenv virtualenv
Suggested packages:
  python-crypto-doc python-cryptography-doc python-cryptography-vectors
  python-dbus-dbg python-dbus-doc python

# Inputs

The following code can be used to generate individuals, where a symbol with $\neg x_i$ corresponds to a symbol $\bar{x}_i$ in the following function definitions. 

In [273]:
input_num = 3
inputs = []
literals = []

for i in string.ascii_letters:
  literals.append(i)
  literals.append('¬' + i)

for i in range(input_num):
  inputs.append(random.sample(literals, len(literals)))

print(inputs[0])
print(inputs[1])
print(inputs[2])

['F', '¬v', '¬S', '¬B', 's', '¬H', 'a', '¬q', 'E', '¬b', '¬M', '¬s', 'L', '¬Z', 'j', 'i', '¬X', 'X', 'U', 'R', 'd', '¬m', 'H', 'O', 'm', '¬P', 'V', 'J', 'p', 'f', '¬F', 'q', '¬j', 't', '¬Q', '¬L', '¬G', 'n', '¬p', 'l', '¬w', 'M', '¬y', '¬Y', '¬W', 'h', '¬C', '¬n', 'P', '¬a', 'y', '¬i', '¬f', '¬E', 'K', 'I', '¬N', 'u', 'B', '¬t', '¬h', '¬x', 'N', 'g', 'C', '¬r', 'D', 'Q', 'A', '¬K', 'Z', '¬O', '¬D', 'c', '¬u', 'z', 'k', 'Y', 'G', 'S', '¬o', 'b', '¬U', '¬c', '¬d', '¬A', 'e', 'v', '¬g', 'W', '¬V', 'w', '¬l', 'o', '¬T', 'x', '¬J', 'r', '¬k', '¬R', 'T', '¬z', '¬I', '¬e']
['¬k', '¬T', '¬z', 'L', '¬V', 'E', 'H', '¬p', 'c', 'U', '¬F', 't', '¬I', '¬v', '¬e', 'W', 'A', '¬o', '¬Q', '¬x', 'T', 'i', 'D', '¬Y', '¬i', '¬P', 'V', '¬J', 'u', 'y', '¬W', '¬s', 'K', '¬L', '¬a', '¬c', 'P', '¬b', 'R', 'C', '¬U', '¬u', '¬S', '¬B', 'q', 'Q', 'a', 'O', 'J', '¬X', '¬r', 'x', '¬R', 'b', 'v', '¬f', 'G', 'g', '¬K', '¬A', 'S', '¬M', 'Y', '¬E', '¬g', 'F', '¬H', 'd', 'M', '¬h', 'f', 'm', 'k', 'p', 's', '¬y', '¬q', 'r

## $\order$

The $\order$ problem, as originally introduced by Goldberg and O’Reilly [1],
is defined as follows.

**Definition ($\order$)**: Let $F=\{J\}$, and $L=\{x_1, \bar{x}_1,\ldots,x_n,\bar{x}_n\}$. The fitness of a tree $X$ is the number of literals $x_i$ for which the positive literal $x_i$ appears before the negative literal $\bar{x}_i$ in the in-order parse of $X$.

$J$ (for "join") is the only available function in this problem, and the fitness of a tree is determined by an in-order parse of its leaf nodes; this reduces the importance of the tree structure in the analysis, making the representation somewhat similar to a variable-length list.

For example, a tree $X$ with in-order parse $(x_1,\bar{x}_4,x_2,\bar{x}_1,x_3,\bar{x}_6)$ has fitness $\order(X):=3$ because $x_1$, $x_2$, and $x_3$ appear before their negations. Any tree that contains all the positive literals and in which each negative literal $\bar{x}_i$ that appears in the tree is preceded by the corresponding positive literal $x_i$ has a fitness of $n$ and is optimal.


### Evolution Strategy for $\order$
**This problem was solved with trees-based programming using the DEAP framework. The following lines of code create the algorithm and the specific parameters that are required to get to the solution. It is important to mention that the presented solution was based on the code of  [4], however important changes were made to make it suitable for this problem. The code is organized as follows:**

1. The creation of the structure of the algorithm


> The creation starts by defining a primitive set, then the operator are  defined as well, and finally the terminal, which in this case are constants. Then, the algorithm is set to have a maximization/minimization approach. Later, the type of individual are created, in this case it is defined as a tree with half with the same depth and the other half with random depth. Finally, the population is defined.

2. The Selection for reproduction
3. The code for variation (crossover and mutation)
4. Selection for survival
> In this part of the code, the evaluation function is defined. It is not completely a selection for survival but it helps the selection process by assining a fitness to the individuals.
5. The main function whichs put together all the previous code to find a solution to the problem
6. The best solution is printed




#### Initialisation

**As mentioned before, the following code creates from scratch the algorithm. This is where the operator is set ('concatenation'), where the constants are defined (taken from literals) and where the type of individuals and population are created.**

In [274]:

import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

rand_val = random.random() * random.random()

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(operator.concat, 2)
pset.addEphemeralConstant("rand"+str(rand_val), lambda: random.sample(literals, 1))

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


#### Stopping Criterion
**The stopping criterion is defined as the number of generations defined in the main function**

#### Selection for Reproduction

**The Selection method is tournament selection**

In [275]:
toolbox.register("select", tools.selTournament, tournsize=3)

#### Variation

**The variation method is composed by crossover and mutation. The crossover is "one point" and the mutation replaces a random node with a tree with the same depth.**

In [276]:
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))

#### Selection for Survival

**It is not exactly a method for survival, but is help to sort the individuals by providing the fitness value. This is the fitness funtion suited for this problem**

In [277]:
def eval_order(individual):
  func = toolbox.compile(expr=individual)
  count = 0
  neg = []
  pos = []
  for var in func:
    if '¬' in var:
      neg.append(var)
    else:
      if ('¬'+var not in neg) and (var not in pos):
        count += 1
        pos.append(var)
  return count,

toolbox.register("evaluate", eval_order)

#### Main Cycle
In the following cell you should implement the main cycle of your evolutionary strategy, this is, putting together all elements defined previously so you can solve the problem.

Once you run the main cycle, the standard output should looks something like this
```
# <Generation/Fitness Evaluation number> <Fitness> <Genotype>
```
Feel free to add extra spaces when needed or use any other format you think will make the output look nicer. But please preserve the order.


**The code below puts together all the components of the algorithm previously defined.**

In [278]:
mRate = 0.2
cRate = 0.1
nGenerations = 120

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cRate, mRate, nGenerations, stats=mstats,
                                   halloffame=hof, verbose=True)
    #print log
    return pop, log, hof 

if __name__ == "__main__":
    pop,log,hof = main()

   	      	                    fitness                     	                      size                     
   	      	------------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max	min	nevals	std     	avg    	gen	max	min	nevals	std    
0  	300   	1.41667	0  	4  	0  	300   	0.903542	4.52667	0  	7  	3  	300   	1.78212
1  	76    	2.19333	1  	5  	0  	76    	0.873282	5.68667	1  	13 	1  	76    	2.18369
2  	95    	2.91   	2  	6  	0  	95    	0.994267	7.03333	2  	17 	1  	95    	2.59979
3  	81    	3.64   	3  	7  	0  	81    	1.00519 	8.94667	3  	19 	1  	81    	2.97049
4  	89    	4.31667	4  	8  	0  	89    	1.16178 	10.5   	4  	25 	1  	89    	3.53318
5  	81    	4.93667	5  	11 	1  	81    	1.32136 	12.3067	5  	25 	1  	81    	4.10519
6  	88    	5.78   	6  	12 	0  	88    	1.64467 	14.7067	6  	31 	1  	88    	4.94712
7  	77    	6.95   	7  	12 	0  	77    	1.54083 	18.44  	7  	31 	1  	77    	4.71661
8  	81    	7.92333	8  	12 	0  	81    	1.68843 

**The next lines of code present the best individual of the population**

In [279]:
best_ind = hof.items[0]
func = toolbox.compile(expr=best_ind)

print('Best Individual Length:', len(toolbox.compile(expr=best_ind)), 'Fitness:',eval_order(best_ind) )
str(func)

Best Individual Length: 90 Fitness: (52,)


"['T', 'p', 'i', 'h', 'X', 'B', 's', 'L', 'o', 'A', 'Q', '¬B', 'S', 'f', 'R', 'H', 'b', '¬A', 'V', 'g', 'v', 'M', 'H', 'C', 'E', 'm', 'w', 'O', 'j', 'u', '¬C', 'n', '¬u', 'd', 'E', 'z', '¬O', 'Z', 'I', 'n', 'y', 'x', 'j', 'R', '¬m', 'q', 'K', 'c', 'U', '¬m', 'e', '¬i', 'F', '¬v', 'w', 'J', '¬z', 'r', 'k', 'h', '¬e', 'N', 'A', 'S', 'i', 'h', '¬X', 'g', '¬f', 'E', 'z', 'O', 'j', 'u', 'W', 'w', 't', '¬F', 'Y', 'Y', '¬o', 'P', 'a', 'G', 'd', 'S', '¬i', 'l', '¬S', 'D']"

## $\mayority$

The $\mayority$ problem, as originally introduced by Goldberg and O’Reilly [1], is defined as follows.

**Definition ($\mayority$)**: Let $F=\{J\}$, and $L=\{x_1,\bar{x}_1,\ldots,x_n,\bar{x}_n\}$. The fitness of a tree $X$ is the number of literals $x_i$ for which the positive literal $x_i$ appears in $X$ at least once, and at least as many times as the corresponding negative literal $\bar{x}_i$.

$J$ (for "join") is the only available function in this problem, and the fitness of a tree is determined by an in-order parse of its leaf nodes; this reduces the importance of the tree structure in the analysis, making the representation somewhat similar to a variable-length list.

For example, a tree with an in-order parse of $(\bar{x}_1,x_1,x_2,x_3,\bar{x}_3,\bar{x}_3)$ would have a fitness $\mayority(X):=2$, as only the literals $x_1$ and $x_2$ are expressed (while $\bar{x}_3$ outnumbers $x_3$ in the tree, and $x_3$ is therefore suppresed). Any optimal, solution, expressing all $n$ positive literals, has a fitness $\mayority(X):=n$.

### Evolution Strategy for $\mayority$
**This problem was solved with trees-based programming using the DEAP framework. The following lines of code create the algorithm and the specific parameters that are required to get to the solution. It is important to mention that the presented solution was based on the code of  [4], however important changes were made to make it suitable for this problem. The code is organized as follows:**

1. The creation of the structure of the algorithm


> The creation starts by defining a primitive set, then the operator are  defined as well, and finally the terminal, which in this case are constants. Then, the algorithm is set to have a maximization/minimization approach. Later, the type of individual are created, in this case it is defined as a tree with half with the same depth and the other half with random depth. Finally, the population is defined.

2. The Selection for reproduction
3. The code for variation (crossover and mutation)
4. Selection for survival
> In this part of the code, the evaluation function is defined. It is not completely a selection for survival but it helps the selection process by assining a fitness to the individuals.
5. The main function whichs put together all the previous code to find a solution to the problem
6. The best solution is printed




#### Initialisation

**As mentioned before, the following code creates from scratch the algorithm. This is where the operator is set ('concatenation'), where the constants are defined (taken from literals) and where the type of individuals and population are created.**


In [281]:
import operator
import math
import random
import pandas as pd
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

rand_val = random.random() * random.random()

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(operator.concat, 2)
pset.addEphemeralConstant("rand90"+str(rand_val), lambda: random.sample(literals, 1))

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


#### Stopping Criterion
**The stopping criterion is defined as the number of generations defined in the main function**


#### Selection for Reproduction

**The Selection method is tournament selection**

In [282]:
toolbox.register("select", tools.selTournament, tournsize=10)

#### Variation

**The variation method is composed by crossover and mutation. The crossover is "one point" and the mutation replaces a random node with a tree with the same depth.**

In [283]:
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))


#### Selection for Survival

**It is not exactly a method for survival, but is help to sort the individuals by providing the fitness value. This is the fitness funtion suited for this problem**

In [284]:
def eval_mayority(individual):
  func = toolbox.compile(expr=individual)

  df = pd.DataFrame(data = {'pos':[0],'neg':[0]}, index = ['a'])
  for elem in func:
    if (elem in df.index.to_list()) or (elem[-1] in df.index.to_list()):
      if '¬' in elem:
        df.loc[elem[-1]]['neg'] = df.loc[elem[-1]]['neg'] + 1 
      else:
        df.loc[elem]['pos'] = df.loc[elem]['pos'] + 1 
    else:
      if '¬' in elem:
        df = pd.concat([pd.DataFrame(data = {'pos':[0],'neg':[0]}, index = [elem[-1]]),df],axis = 0)
        df.loc[elem[-1]]['neg'] = df.loc[elem[-1]]['neg'] + 1 
      else:
        df = pd.concat([pd.DataFrame(data = {'pos':[0],'neg':[0]}, index = [elem]),df],axis = 0)
        df.loc[elem]['pos'] = df.loc[elem]['pos'] + 1
    
  res = df['pos'] >= df['neg'] 
  return res.sum(),

toolbox.register("evaluate", eval_mayority)


#### Main Cycle
In the following cell you should implement the main cycle of your evolutionary strategy, this is, putting together all elements defined previously so you can solve the problem.

Once you run the main cycle, the standard output should looks something like this
```
# <Generation/Fitness Evaluation number> <Fitness> <Genotype>
```
Feel free to add extra spaces when needed or use any other format you think will make the output look nicer. But please preserve the order.


**The code below puts together all the components of the algorithm previously defined. NOTE: THE CODE TAKES AROUND 2min AND 30sec TO GET TO THE OPTIMAL**

In [286]:
cRate = 0.0
mRate = 0.1
nGenerations = 90


def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cRate, mRate,nGenerations, stats=mstats,
                                   halloffame=hof, verbose=True)
    #print log
    return pop, log, hof 

if __name__ == "__main__":
    pop,log,hof = main()

   	      	                    fitness                     	                      size                     
   	      	------------------------------------------------	-----------------------------------------------
gen	nevals	avg 	gen	max	min	nevals	std     	avg    	gen	max	min	nevals	std    
0  	300   	2.33	0  	5  	0  	300   	0.935111	4.52667	0  	7  	3  	300   	1.78212
1  	27    	3.77	1  	8  	1  	27    	0.793998	6.28667	1  	13 	3  	27    	1.96244
2  	43    	4.94333	2  	8  	1  	43    	1.05204 	8.48667	2  	19 	1  	43    	2.96476
3  	21    	6.47667	3  	10 	2  	21    	1.16739 	12.4467	3  	21 	1  	21    	2.97105
4  	28    	7.89667	4  	10 	1  	28    	0.875208	13.2933	4  	23 	3  	28    	1.9237 
5  	24    	8.42333	5  	10 	1  	24    	1.1154  	14.9333	5  	23 	1  	24    	3.11056
6  	32    	9.67   	6  	11 	1  	32    	1.1638  	18.5933	6  	25 	1  	32    	2.52612
7  	35    	9.91   	7  	12 	3  	35    	0.817659	19.1467	7  	25 	5  	35    	1.96769
8  	31    	10.3467	8  	12 	5  	31    	0.916418	20.2733	

**The next lines of code present the best individual of the population**

In [287]:
best_ind = hof.items[0]
func = toolbox.compile(expr=best_ind)

print('Best Individual Length:', len(toolbox.compile(expr=best_ind)), 'Fitness:',eval_mayority(best_ind))
str(func)

Best Individual Length: 60 Fitness: (52,)


"['k', 'f', 'C', 'F', 'U', 'A', 'h', 'V', 's', 'J', 'c', 'O', 'T', 'E', 'K', 'R', 'q', 'i', 'e', 'b', 'm', 'g', 'x', 'l', 'w', '¬d', 'g', 'P', 'S', 'z', 'r', 'p', 'c', 't', 'o', 'Y', 'G', 'v', 'X', 'H', 'L', 'z', 'D', 'B', 'B', 'u', 'j', '¬G', 'I', 'S', 'v', 'Q', 'I', 'Z', 'd', 'N', 'n', 'M', 'y', 'W']"

## $\sorting$

Wagner et al. analyzed the performance of GP for the problem, aiming to investigate the differences between different bloat control mechanisms for GP [2, 3]. For GP, the measures of sortedness were adapted to deal with incomplete permutations of the literal set.

**Definition ($\sorting$)**: Let $F=\{J\}$, and $L=\{1,2,\ldots,n\}$. The fitness of a tree $X$ is computed by deriving a sequence $\pi$ of symbols based on their first appearance in the in-order parse of $X$, and considering one of the following five measures of sortedness of this sequence.

- $\inv(\pi)$: Number of pairs of adjacent elements in the correct order (maximise to sort), with $\inv(\pi):=0.5$ if $|\pi|:=1$.
- $\ham(\pi)$: Number of elements in correct position (maximise to sort).
- $\run(\pi)$: Number of maximal sorted blocks (minimise to sort), plus the number of missing elements $n-|\pi|$, with $\run(\pi):=n+1$ if $|\pi|=0$.
- $\las(\pi)$: Length of longest ascending sequence (maximise to sort).
- $\exc(\pi)$: Smallest number of exchanges needed to sort the sequence (minimise to sort), plus $1+n-|\pi|$ if $|\pi|< n$

$J$ (for "join") is the only available function in this problem, and the fitness of a tree is determined by an in-order parse of its leaf nodes drawn from a totally ordered set of terminals $L$. This reduces the importance of the tree structure in the analysis, making the representation somewhat similar to a variable-length list.

Thus, for $n:=5$, the fitness of a tree with an in-order parse of $(1,2,1,4,5,4,3)$, and hence $\pi=\{1,2,4,5,3\}$ is $\inv(\pi):=3$, $\ham(\pi):=2$, $\run(\pi):=2$, $\las(\pi):=4$, and $\exc(\pi):=2$. The fitness value of optimal trees for the $\inv$, $\ham$, and $\las$ measures is $n$, while for the $\run$ and $\exc$ measures it is $0$.



### Evolution Strategy for $\sorting$

**This problem was solved with trees-based programming using the DEAP framework. The following lines of code create the algorithm and the specific parameters that are required to get to the solution. It is important to mention that the presented solution was based on the code of  [4], however important changes were made to make it suitable for this problem. The code is organized as follows:**

1. The creation of the structure of the algorithm


> The creation starts by defining a primitive set, then the operator are  defined as well, and finally the terminal, which in this case are constants. Then, the algorithm is set to have a maximization/minimization approach. Later, the type of individual are created, in this case it is defined as a tree with half with the same depth and the other half with random depth. Finally, the population is defined.

2. The Selection for reproduction
3. The code for variation (crossover and mutation)
4. Selection for survival
> In this part of the code, the evaluation function is defined. It is not completely a selection for survival but it helps the selection process by assining a fitness to the individuals.
5. The main function whichs put together all the previous code to find a solution to the problem
6. The best solution is printed


**NOTE: In order to avoid repetitions of text, the sections Intialisation, Stopping criterion, Selection for Reproduction, Variation, Survival Method and main code were removed because all of this information is the same as the Order and Mayority problems. Instead, the sections are specific for each one of the types of sorting functions. I hope this is not an inconvinient.** 

#### INV

**The following code implements the INV function**

In [288]:
import operator
import math
import random
import pandas as pd
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


n = 10
l = np.arange(1,n)
numbers = list(l)
numbers

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [289]:
rand = random.random() * random.random()

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(operator.concat, 2)
pset.addEphemeralConstant("rand410"+str(rand), lambda: random.sample(numbers, 1))

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


def inv(individual):
  x1 = toolbox.compile(expr=individual)
  count = 0
  pi = []
  for i in x1:
    if i not in pi:
      pi.append(i)

  if len(pi)>1:
    for i in range (len(pi)-1):
      if pi[i] < pi[i+1]:
        count += 1
  else:
    count = 0.5
  return count,

toolbox.register("evaluate", inv)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.0, 0.9, 90, stats=mstats,
                                   halloffame=hof, verbose=True)
    #print log
    return pop, log, hof 

if __name__ == "__main__":
    pop,log,hof = main()

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	                    fitness                     	                      size                     
   	      	------------------------------------------------	-----------------------------------------------
gen	nevals	avg  	gen	max	min	nevals	std     	avg    	gen	max	min	nevals	std    
0  	300   	0.755	0  	3  	0  	300   	0.633094	4.56667	0  	7  	3  	300   	1.82178
1  	264   	1.40333	1  	3  	0  	264   	0.814855	6.97333	1  	13 	1  	264   	3.22066
2  	260   	2.15667	2  	4  	0  	260   	0.968911	10.2267	2  	19 	1  	260   	4.00524
3  	271   	2.83333	3  	6  	0  	271   	1.15205 	13.7067	3  	25 	1  	271   	5.12581
4  	273   	3.64333	4  	6  	0  	273   	1.25278 	17.8067	4  	31 	1  	273   	5.82431
5  	263   	4.315  	5  	6  	0  	263   	1.44509 	20.72  	5  	35 	1  	263   	7.0693 
6  	274   	4.71667	6  	7  	0  	274   	1.34464 	24.56  	6  	41 	1  	274   	7.38059
7  	275   	4.97333	7  	7  	0  	275   	1.31756 	26.9267	7  	47 	1  	275   	8.258  
8  	270   	5.23667	8  	7  	0  	270   	1.33815 	29.

In [290]:
best_ind = hof.items[0]
func = toolbox.compile(expr=best_ind)

print('Best Individual Length:', len(toolbox.compile(expr=best_ind)), 'Fitness:',inv(best_ind))
print('Individual body',str(func))

pi_t = []
for i in func:
  if i not in pi_t:
    pi_t.append(i)
print('Pi:',pi_t)

Best Individual Length: 80 Fitness: (8,)
Individual body [1, 2, 3, 4, 5, 6, 7, 8, 9, 7, 1, 9, 3, 4, 8, 8, 4, 1, 6, 2, 4, 2, 5, 8, 9, 2, 7, 2, 5, 4, 9, 2, 4, 9, 3, 1, 1, 5, 8, 3, 1, 9, 4, 1, 3, 1, 2, 5, 6, 6, 2, 3, 6, 9, 1, 6, 1, 8, 1, 1, 9, 5, 4, 7, 9, 8, 5, 4, 6, 1, 6, 2, 6, 1, 5, 5, 6, 8, 6, 2]
Pi: [1, 2, 3, 4, 5, 6, 7, 8, 9]


#### HAM

**The following codes implements the ham function**

In [291]:
rand = random.random() * random.random()

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(operator.concat, 2)
pset.addEphemeralConstant("rand510"+str(rand), lambda: random.sample(numbers, 1))

creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


def ham(individual, y1):
  """
  Returns the hamming distance
  """
  x1 = toolbox.compile(expr=individual)
  count = 0
  if len(x1) > len(y1):
    for i in range (len(y1)):
      if x1[i] == y1[i]:
        count += 1
  else:
    for i in range (len(x1)):
      if x1[i] == y1[i]:
        count += 1 
  return count,

toolbox.register("evaluate", ham, y1 = numbers)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.0, 0.9, 20, stats=mstats,
                                   halloffame=hof, verbose=True)
    #print log
    return pop, log, hof 

if __name__ == "__main__":
    pop,log,hof = main()

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	                        fitness                         	                      size                     
   	      	--------------------------------------------------------	-----------------------------------------------
gen	nevals	avg     	gen	max	min	nevals	std     	avg    	gen	max	min	nevals	std    
0  	300   	0.326667	0  	2  	0  	300   	0.516355	4.56667	0  	7  	3  	300   	1.82178
1  	266   	0.713333	1  	3  	0  	266   	0.719599	6.28667	1  	13 	1  	266   	3.30018
2  	265   	1.16333 	2  	4  	0  	265   	0.971248	9.32   	2  	19 	1  	265   	4.29313
3  	274   	1.81    	3  	5  	0  	274   	1.21129 	13.02  	3  	25 	1  	274   	5.60115
4  	273   	2.43667 	4  	5  	0  	273   	1.34387 	17.22  	4  	31 	1  	273   	5.88486
5  	267   	2.85    	5  	5  	0  	267   	1.55804 	18.9333	5  	37 	1  	267   	6.37042
6  	282   	3.52333 	6  	7  	0  	282   	1.56933 	20.9867	6  	41 	1  	282   	7.1833 
7  	275   	3.78    	7  	7  	0  	275   	1.79581 	21.5933	7  	41 	1  	275   	7.60228
8  	272   	4.07    	8

In [292]:
best_ind = hof.items[0]
func = toolbox.compile(expr=best_ind)

print('Best Individual Length:', len(toolbox.compile(expr=best_ind)), 'Fitness:',ham(best_ind, numbers))
print('Individual Body')
pi_t = []
for i in func:
  if i not in pi_t:
    pi_t.append(i)
print('Pi:',pi_t)

Best Individual Length: 16 Fitness: (9,)
Individual Body
Pi: [1, 2, 3, 4, 5, 6, 7, 8, 9]


#### RUN

**The following code implements the RUN function**

In [294]:
rand = random.random()*random.random()

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(operator.concat, 2)
pset.addEphemeralConstant("rand210"+str(rand), lambda: random.sample(numbers, 1))

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=0, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


def run(individual, numbers):
  x1 = toolbox.compile(expr=individual)
  counts = []
  count = 0
  pi = []
  for i in x1:
    if i not in pi:
      pi.append(i)

  if len(pi) == len(numbers):
    for i in range (len(pi)-1):
      if pi[i] < pi[i+1]:
        count += 1
      elif pi[i] > pi[i+1]:
        counts.append(count)
        count = 1
      if i == len(pi)-2:
        counts.append(count)
    result = len(counts)
  else:
    result = 11
  return result-1,

toolbox.register("evaluate", run, numbers = numbers)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.9, 0.9, 30, stats=mstats,
                                   halloffame=hof, verbose=True)
    #print log
    return pop, log, hof 

if __name__ == "__main__":
    pop,log,hof = main()

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	                  fitness                  	                     size                     
   	      	-------------------------------------------	----------------------------------------------
gen	nevals	avg	gen	max	min	nevals	std	avg    	gen	max	min	nevals	std   
0  	300   	10 	0  	10 	10 	300   	0  	2.80667	0  	7  	1  	300   	2.2217
1  	297   	10 	1  	10 	10 	297   	0  	4.38   	1  	13 	1  	297   	3.22939
2  	297   	10 	2  	10 	10 	297   	0  	6.15333	2  	17 	1  	297   	3.78636
3  	293   	10 	3  	10 	10 	293   	0  	7.41333	3  	21 	1  	293   	4.43499
4  	297   	10 	4  	10 	10 	297   	0  	8.73333	4  	29 	1  	297   	5.03411
5  	295   	9.98333	5  	10 	5  	295   	0.288194	9.6    	5  	27 	1  	295   	5.5845 
6  	299   	9.96333	6  	10 	4  	299   	0.449432	10.48  	6  	33 	1  	299   	6.57036
7  	298   	9.87   	7  	10 	4  	298   	0.844452	12.1467	7  	45 	1  	298   	8.06919
8  	296   	9.36667	8  	10 	2  	296   	1.84902 	16.14  	8  	47 	1  	296   	10.8453
9  	295   	7.78   	9  	10 	2  	2

In [295]:
best_ind = hof.items[0]
func = toolbox.compile(expr=best_ind)

print('Best Individual Length:', len(toolbox.compile(expr=best_ind)), 'Fitness:',run(best_ind,numbers))
print('Individual body',str(func))

pi_t = []
for i in func:
  if i not in pi_t:
    pi_t.append(i)
print('Pi:',pi_t)

Best Individual Length: 56 Fitness: (0,)
Individual body [1, 2, 2, 3, 4, 4, 1, 5, 3, 6, 1, 1, 7, 4, 7, 4, 8, 7, 4, 2, 3, 2, 3, 2, 5, 8, 2, 6, 5, 2, 3, 7, 7, 9, 7, 7, 2, 8, 9, 8, 9, 5, 7, 9, 1, 9, 9, 5, 6, 5, 3, 8, 1, 9, 7, 1]
Pi: [1, 2, 3, 4, 5, 6, 7, 8, 9]


#### LAS
**The following code implements the LAS function**

In [296]:
rand = random.random() * random.random()

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(operator.concat, 2)
pset.addEphemeralConstant("rand610"+str(rand), lambda: random.sample(numbers, 1))

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


def las(individual):
  x1 = toolbox.compile(expr=individual)
  counts = []
  count = 1
  if len(x1)>1:
    for i in range (len(x1)-1):
      if x1[i] < x1[i+1]:
        count += 1
      elif x1[i] > x1[i+1]:
        counts.append(count)
        count = 1
      if i == len(x1)-2:
        counts.append(count)
  else:
    counts.append(count)
  return max(counts),
  
toolbox.register("evaluate", las)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.1, 0.9, 20, stats=mstats,
                                   halloffame=hof, verbose=True)
    #print log
    return pop, log, hof 

if __name__ == "__main__":
    pop,log,hof = main()

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	                    fitness                     	                      size                     
   	      	------------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max	min	nevals	std     	avg    	gen	max	min	nevals	std    
0  	300   	1.71667	0  	4  	1  	300   	0.650427	4.56667	0  	7  	3  	300   	1.82178
1  	272   	2.28   	1  	4  	1  	272   	0.788416	7.13333	1  	15 	1  	272   	3.29579
2  	260   	2.80667	2  	5  	1  	260   	0.873282	9.66   	2  	19 	1  	260   	4.06666
3  	270   	3.21   	3  	5  	1  	270   	0.982802	11.9133	3  	29 	1  	270   	5.22614
4  	277   	3.49333	4  	6  	1  	277   	1.0376  	14.6   	4  	35 	1  	277   	6.23966
5  	269   	3.88   	5  	6  	1  	269   	1.17994 	17.5   	5  	41 	1  	269   	7.16496
6  	276   	4.34667	6  	8  	1  	276   	1.15751 	19.84  	6  	49 	1  	276   	7.6027 
7  	270   	4.58   	7  	8  	1  	270   	1.38212 	21.0933	7  	55 	1  	270   	8.89745
8  	273   	5.00667	8  	8  	1  	273   	1.63502 

In [297]:
best_ind = hof.items[0]
func = toolbox.compile(expr=best_ind)

print('Best Individual Length:', len(toolbox.compile(expr=best_ind)), 'Fitness:',las(best_ind))
print('Individual Body')
pi_t = []
for i in func:
  if i not in pi_t:
    pi_t.append(i)
print('Pi:',pi_t)

Best Individual Length: 17 Fitness: (9,)
Individual Body
Pi: [1, 2, 3, 4, 5, 6, 7, 8, 9]


#### EXC
**The following code implements the EXC function**

In [299]:
rand = random.random() * random.random()

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(operator.concat, 2)
pset.addEphemeralConstant("rand710"+str(rand), lambda: random.sample(numbers, 1))

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


def exc(individual, numbers):
  x1 = toolbox.compile(expr=individual)
  count = 0
  pi = []
  for i in x1:
    if i not in pi:
      pi.append(i)

  if len(pi) == len(numbers):
    for i in range (len(numbers)):
      if pi[i] != numbers[i]:
        count += 1
  else:
    count = 10
  return count,

toolbox.register("evaluate", exc, numbers = numbers)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.9, 0.9, 20, stats=mstats,
                                   halloffame=hof, verbose=True)
    #print log
    return pop, log, hof 

if __name__ == "__main__":
    pop,log,hof = main()

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	                  fitness                  	                      size                     
   	      	-------------------------------------------	-----------------------------------------------
gen	nevals	avg	gen	max	min	nevals	std	avg    	gen	max	min	nevals	std    
0  	300   	10 	0  	10 	10 	300   	0  	4.56667	0  	7  	3  	300   	1.82178
1  	300   	10 	1  	10 	10 	300   	0  	5.8    	1  	15 	1  	300   	3.1496 
2  	299   	10 	2  	10 	10 	299   	0  	6.88   	2  	21 	1  	299   	4.10677
3  	298   	10 	3  	10 	10 	298   	0  	7.58   	3  	23 	1  	298   	4.58079
4  	299   	9.98333	4  	10 	7  	299   	0.207498	8.62667	4  	27 	1  	299   	5.25046
5  	299   	9.96333	5  	10 	7  	299   	0.297751	10.3333	5  	33 	1  	299   	6.85339
6  	297   	9.82333	6  	10 	7  	297   	0.631497	14.0067	6  	45 	1  	297   	9.62981
7  	297   	9.33667	7  	10 	5  	297   	1.15902 	22.5867	7  	65 	1  	297   	14.4928
8  	296   	8.32   	8  	10 	4  	296   	1.57404 	36.24  	8  	105	1  	296   	15.8454
9  	298   	7.36333	

In [300]:
best_ind = hof.items[0]
func = toolbox.compile(expr=best_ind)

print('Best Individual Length:', len(toolbox.compile(expr=best_ind)), 'Fitness:',exc(best_ind, numbers))
print('Individual body',str(func))

pi_t = []
for i in func:
  if i not in pi_t:
    pi_t.append(i)
print('Pi:',pi_t)

Best Individual Length: 54 Fitness: (0,)
Individual body [1, 2, 3, 4, 5, 3, 6, 4, 7, 7, 8, 9, 6, 1, 2, 9, 7, 5, 4, 2, 3, 6, 6, 2, 9, 5, 9, 4, 4, 2, 1, 8, 6, 2, 2, 6, 3, 8, 3, 2, 5, 5, 4, 4, 8, 2, 1, 2, 4, 5, 6, 1, 2, 3]
Pi: [1, 2, 3, 4, 5, 6, 7, 8, 9]


#### Conclusion

**All the problems were solved by the algorithms presented in this Notebook. The approach used was the tree-based programming using DEAP framework. Most of the solutions were maximization problems, however only two of them were minimization (Sorting). In some of the problems the parameters for crossover and mutations were different because they produced different behaviors on the solutions. Finally, the code that took more time to run (because of complexity and fitness evaluation) was "Mayority", and the code that took less generations was LAS for the Sorting problem. Each one of the problem presented a challenge but fortunately all of them could be solved with almost the same approach.**

# References

<ol type = "1">
<li>Goldberg, D.E., O’Reilly, U.: Where does the good stuff go, and why? how contextual semantics influences program structure in simple genetic programming. In: Proceedings of Genetic Programming, First European Workshop (EuroGP 1998), pp. 16–36 (1998).</li>
<li>Wagner, M., Neumann, F.: Parsimony pressure versus multi-objective
optimization for variable length representations. In: Proceedings of the
12th International Conference on Parallel Problem Solving from Nature
(PPSN 2012), pp. 133–142 (2012).</li>
<li>Wagner, M., Neumann, F., Urli, T.: On the performance of different genetic
programming approaches for the SORTING problem. Evolutionary
Computation 23(4), 583–609 (2015).</li>
<li> fmder. (2014). symbreg.py. Novemeber, 2nd, de Github Sitio web: https://github.com/DEAP/deap/blob/454b4f65a9c944ea2c90b38a75d384cddf524220/examples/gp/symbreg.py</li>
</ol>
